In [ ]:
!pip install pandas matplotlib seaborn

In [ ]:
!pip install tensorflow

In [ ]:
!pip install transformers datasets evaluate accelerate

In [2]:
from datasets import load_dataset, DatasetDict
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
import nltk
from nltk.util import ngrams
from collections import Counter

In [10]:
dataset = load_dataset('sst')
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})

In [34]:
dataset.map(lambda examples: {'label': int(examples['label'])}, batched=True)

Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

## 1. Know the `cola` dataset

In [14]:
train_df = dataset["train"].to_pandas()
validation_df = dataset["validation"].to_pandas()
test_df = dataset["test"].to_pandas()

## Save the first `100` rows to a `csv` files

In [28]:
first_100_rows = train_df.head(100)
first_100_rows.to_csv('../data/sst_example.csv', index=False)

In [15]:
train_df.head(10)

,sentence,label,tokens,tree
0,The Rock is destined to be the 21st Century 's...,0.69444,The|Rock|is|destined|to|be|the|21st|Century|'s...,70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|5...
1,The gorgeously elaborate continuation of `` Th...,0.83333,The|gorgeously|elaborate|continuation|of|``|Th...,71|70|69|69|67|67|66|64|63|62|62|61|61|58|57|5...
2,Singer\/composer Bryan Adams contributes a sle...,0.62500,Singer\/composer|Bryan|Adams|contributes|a|sle...,72|71|71|70|68|68|67|67|66|63|62|62|60|60|58|5...
3,You 'd think by now America would have had eno...,0.50000,You|'d|think|by|now|America|would|have|had|eno...,36|35|34|33|33|32|30|29|27|26|25|24|23|23|22|2...
4,Yet the act is still charming here .,0.72222,Yet|the|act|is|still|charming|here|.,15|13|13|10|9|9|11|12|10|11|12|14|14|15|0
5,Whether or not you 're enlightened by any of D...,0.83333,Whether|or|not|you|'re|enlightened|by|any|of|D...,62|59|59|58|57|56|55|54|53|52|52|51|50|49|44|4...
6,Just the labour involved in creating the layer...,0.87500,Just|the|labour|involved|in|creating|the|layer...,41|40|40|39|38|37|35|34|34|33|31|31|30|28|28|2...
7,Part of the charm of Satin Rouge is that it av...,0.72222,Part|of|the|charm|of|Satin|Rouge|is|that|it|av...,34|33|31|31|30|29|29|27|26|25|23|22|22|21|19|1...
8,a screenplay more ingeniously constructed than...,0.83333,a|screenplay|more|ingeniously|constructed|than...,16|15|15|14|13|10|10|11|12|11|12|13|14|17|16|17|0
9,`` Extreme Ops '' exceeds expectations .,0.73611,``|Extreme|Ops|''|exceeds|expectations|.,13|11|11|10|8|8|9|9|10|12|12|13|0


In [16]:
test_df.describe()

,label
count,2210.000000
mean,0.511444
std,0.256466
min,0.000000
25%,0.277780
50%,0.500000
75%,0.757815
max,1.000000


In [17]:
test_df.shape

(2210, 4)

In [38]:
dataset["train"][0]

{'sentence': "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .",
 'label': 0.0,
 'tokens': "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.",
 'tree': '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'}